In [1]:
import pandas as pd
import string
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
class word_by_duc:
  def __init__(self):
    persian_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
    self.punctuations_list = string.punctuation + persian_punctuations
  def fit(self, data, words, col_name_text, col_name_label):
    self.col_name_text = col_name_text
    self.col_name_label = col_name_label
    self.data = data
    self.words = words
    self.df_result = pd.DataFrame()
    self.name_labels = list(set(self.data[self.col_name_label]))


    self.data[self.col_name_text] = self.data[self.col_name_text].apply(self._remove_punctuations)
    self.data[self.col_name_text] = self.data[self.col_name_text].apply(lambda sent : sent.split())
    self.data[self.col_name_text] = self.data[self.col_name_text].apply(self._clean_words)

    self._create_all_words()
    self._create_target_words()


    self.df_result["words"] = self.target_words
    self._add_columns()
    self._fit_matrix()
    self.df_result = self.df_result.set_index("words")
    return self.df_result


  def _remove_punctuations(self, text):
    translator = str.maketrans('', '', self.punctuations_list)
    return text.translate(translator)

  def _clean_words(self, words):
    for index in range(len(words)):
      words[index] = words[index].rstrip()
      words[index] = words[index].lstrip()
    return words

  def _add_columns(self):
    for col in self.name_labels:
      self.df_result[col] = None

  def _fit_matrix(self):
    for category in self.name_labels:
      repeat = []
      for index in range(self.df_result.shape[0]):
        repeat.append(self._get_repeat_words(self.df_result.iloc[index]["words"], category))
      self.df_result[category] = repeat

  def _get_repeat_words(self, word, category):
    count = 0
    for index in range(self.data.shape[0]):
      if word in self.data.iloc[index][self.col_name_text] and self.data.iloc[index][self.col_name_label] ==  category:
        count += 1
    return count

  def _create_all_words(self):
    self.all_words = list()
    for index in range(0, self.data.shape[0]):
      for word in self.data.iloc[index][self.col_name_text]:
        if word not in self.all_words:
          self.all_words.append(word)

  def _create_target_words(self):
    self.target_words = list()
    for word in self.words:
      if word in self.all_words:
        if word not in self.target_words:
          self.target_words.append(word)
          print(f"---- {word} added to list")
        else:
          print(f"---- {word} not added to list")



In [43]:
df = pd.read_csv("/content/drive/MyDrive/get news/ettemad/ettemad_all.csv", lineterminator='\n')
df = df.drop(["Unnamed: 0"], axis=1)
df = df[["title", "label"]]
df.head()

,title,label
0,از «احسانو» چه خبر؟,culture-media
1,مراسم تشییع پیکر حسین زمان برگزار شد,culture-media
2,تصویر | ساعت حرم امام رضا (ع) که به سفارش مظفر...,culture-media
3,تصاویر | مجازات زنان و مردان در ملاء عام دوره ...,culture-media
4,وقتی از شهرت «بنکسی» سوءاستفاده می شود+ عکس,culture-media


In [44]:
model = word_by_duc()
words = ["مجازات", "آمریکا", "استقلال", "حسین", "عراق", "روسیه", "زن"]
model.fit(df, words , "title", "label")

---- مجازات added to list
---- آمریکا added to list
---- استقلال added to list
---- حسین added to list
---- عراق added to list
---- روسیه added to list
---- زن added to list


,culture-media,sports,politics,social,economy
words,,,,,
مجازات,2,0,0,1,0
آمریکا,1,2,40,3,11
استقلال,0,160,0,0,0
حسین,9,2,1,0,3
عراق,1,0,5,2,16
روسیه,2,1,19,0,5
زن,9,9,9,25,2
